In [283]:
import psycopg2
import pandas as pd

In [286]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Предварительный анализ данных

## Задание 3.1

*Сколько вакансий есть в базе?*

In [287]:
#Код запроса 3.1
query3_1 = f'''select 
            count(id)
            from vacancies
        '''

In [288]:
#Результат запроса 3.1
df = pd.read_sql_query(query3_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1387615049.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query3_1, connection)


,count
0,49197


## Задание 3.2

*Сколько работодателей?*

In [289]:
#Код запроса 3.2
query3_2 = f'''select 
            count(id)
            from employers
        '''

In [290]:
#Результат запроса 3.2
df = pd.read_sql_query(query3_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\3013998970.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query3_2, connection)


,count
0,23501


## Задание 3.3

*Сколько регионов?*

In [291]:
#Код запроса 3.3
query3_3 = f'''select 
            count(id)
            from areas
        '''

In [292]:
#Результат запроса 3.3
df = pd.read_sql_query(query3_3, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\3236700621.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query3_3, connection)


,count
0,1362


## Задание 3.4

*Сколько сфер деятельности?*

In [293]:
#Код запроса 3.4
query3_4 = f'''select 
            count(id)
            from industries
        '''

In [294]:
#Результат запроса 3.4
df = pd.read_sql_query(query3_4, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\2293237446.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query3_4, connection)


,count
0,294


## Общий вывод по блоку 3:

*Имеем базу данных с вакансиями на сайте HeadHunter. База данныъ разделена на 5 частей и включает в себя:*

1. Данные по размещённым вакансиям;
2. Данные по регионам, соответствующих вакансий;
3. Даннае о работадателе, размещающем вакансию;
4. Данные о сфере деятельности компании-работодателя;
5. Дополнительная таблица, с ключами (Id) работодателя и сферы деятельности.


*После анализа количества данных, можем сделать следующие выводы:
*Всего в БД содержится:*

3.1. 49197 вакансий;

3.2 23501 вакансия;

3.3. 1362 региона;

3.4. 214 сфер деятельности.

# Детальный анализ вакансий

## Задание 4.1

*Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).*

*Выберите пятёрку лидеров по количеству вакансий:*

In [295]:
#Код запроса 4.1
query4_1 = f'''select
            count(v.id) cnt,                            -- считаем количество вакансий 
            a.name area                                 -- имя региона
            from areas a
            join vacancies v on a.id = v.area_id        -- соединение таблиц по ключевому параметру
            group by a.name                             -- группировка данных по регионам
            order by count(v.id) desc                   -- сортировка данных по количеству в порядке убывания
            limit 5
        '''

In [296]:
#Результат запроса 4.1
df = pd.read_sql_query(query4_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1180605620.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query4_1, connection)


,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


## Задание 4.2

*Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?*

In [297]:
#Код запроса 4.2
query4_2 = f'''select
            count(id)                                                       -- запрос количества ваканчий
            from vacancies v  
            where v.salary_from is not NULL or v.salary_to is not NULL      -- отфильтровываем вакансии, где зарплата не указана
        '''

In [298]:
#Результат запроса 4.2
df = pd.read_sql_query(query4_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\595047708.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query4_2, connection)


,count
0,24073


## Задание 4.3

*Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.*

In [299]:
#Код запроса 4.3
query4_3 = f'''select
            round(avg(salary_from)) avg_from,       -- находим среднее значение уровня зарплаты по нижней гарнице
            round(avg(salary_to)) avg_to            -- находим среднее значение уровня зарплаты по верхней гарнице
            from vacancies
        '''

In [300]:
#Результат запроса 4.3
df = pd.read_sql_query(query4_3, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1291227656.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query4_3, connection)


,avg_from,avg_to
0,71065.0,110537.0


## Задание 4.4

*Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Какая пара находится на втором месте по популярности?*

In [301]:
#Код запроса 4.4
query4_4 = f'''select
            count(id),                              -- запрос количества вакансий
            schedule,                               -- запрос типа рабочего графика         
            employment                              -- запрос типа трудосустройства        
            from vacancies
            group by schedule, employment           -- группировка по типам schedule и employment
            order by count(id) desc                 -- сортировка в порядке убывания
        '''

In [302]:
#Результат запроса 4.4
df = pd.read_sql_query(query4_4, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1832738517.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query4_4, connection)


,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


## Задание 4.5

*Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.*

In [303]:
#Код запроса 4.5
query4_5 = f'''select
            count(id),                              -- запрос количества вакансий
            experience                              -- запрос строки опыта работы
            from vacancies
            group by experience                     -- группировка по строке опыта работы
            order by count(id)                      -- сортировка по количеству в порядке возрастания
        '''

In [304]:
#Результат запроса 4.5
df = pd.read_sql_query(query4_5, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1845270137.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query4_5, connection)


,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


## Общий вывод по блоку 4:

*После анализа вакансий можно сделать следующие выводы:*

4.1. В ТОП 5 по количесву вакансий входят следующие города:
    - Москва;
    - Санкт-Петербург;
    - Минск;
    - Новосибирск;
    - Алматы.

4.2. Количество вакансий с указанием зарплаты составляет 24073, что составляет примерно 50% от всех вакансий.

4.3. Средняя зарплата по РФ и странгам СНГ составляет от 71065 до 110537.

4.4. *Наибольшей популярностью пользуются вакансии с типом трудоустрйоства  ***Полная занятость*** и графиками работы ***Полный день, удалённая работа****

4.5. *Наиболее распространенные вакансии по уровню опыта работы: ***от 1     года до 3 лет***, наименее распространенные: ***Более 6 лет****



# 5. Анализ работодателей

## Задание 5.1

*Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.*

In [305]:
#Код запроса 5.1
query5_1 = f'''select
            count(v.id),                                                    -- запрос количества вакансий
            e.name имя_работодателя                                         -- запрос имени компании-работодателя
            from vacancies v
            join employers e on v.employer_id = e.id                        -- соединение таблиц вакансия-работодатель
            group by e.name                                                 -- группировка по имени компании-работодателя
            order by count(v.id) desc                                       -- сортировка по уоличеству вакансий от каждой компании-работодателя в порядке убывания
            limit 5
        '''

In [306]:
#Результат запроса 5.1
df = pd.read_sql_query(query5_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\3711006025.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5_1, connection)


,count,имя_работодателя
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


## Задание 5.2

*Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.*

*Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.*

In [309]:
#Код запроса 5.2
query5_2 = f'''select
            a.name area,                                                        -- запрос региона
            count(e.id) emp_count,                                              -- запрос количества компаний-работодателя
            count(v.id) vac_count                                               -- запрос количества вакансий
            from public.areas a 
            left join vacancies v on v.area_id = a.id                           -- соединение таблиц (left join - ) регион-вакансия, регион - работодатель (по id региона)
            left join employers e on e.area = a.id
            where v.id is null                                                  -- фильтрация объединенной таблицы
            group by a.name                                                     -- группировка по имени региона
            order by emp_count desc                                             -- сортировка в порядке убывания
            limit 10
        '''

In [310]:
#Результат запроса 5.2
df = pd.read_sql_query(query5_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\2414183511.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5_2, connection)


,area,emp_count,vac_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
5,Ростовская область,18,0
6,Азербайджан,17,0
7,Нижегородская область,16,0
8,Республика Татарстан,16,0
9,Узбекистан,15,0


## Задание 5.3

*Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.*

*Выберите максимальное значение из получившегося списка.*

In [311]:
#Код запроса 5.3
query5_3 = f'''select
            e.name,                                                                     -- Запрос имени компании-работодателя
            count(distinct v.area_id) area_count                                        -- Запрос количества уникальных имен регионов
            from vacancies v
            left join areas a on v.area_id = a.id
            left join employers e on v.employer_id = e.id                               -- Соединение таблиц вакансия- регион, вакансия-работодатель
            group by e.name                                                             -- группировка по имени компании-работодателя
            order by count(a.id) desc
            limit 10
        '''

In [312]:
#Результат запроса 5.3
df = pd.read_sql_query(query5_3, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\2125719512.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5_3, connection)


,name,area_count
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24
5,ИК СИБИНТЕК,46
6,МТС,55
7,DataArt,2
8,Совкомбанк Технологии,26
9,Первый Бит,43


## Задание 5.4

*Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.*

*Введите количество, которое вернул запрос, в поле ниже.*

In [313]:
#Код запроса 5.4
query5_4 = f'''select
            count(e.id)                                                                     -- Запрос количества работодателей
            from public.employers e
            left join public.employers_industries ei on e.id  = ei.employer_id       
            left join public.industries i on i.id = ei.industry_id                          -- Соединение таблиц работодатель - вспомогательная таблица, работодатель - сфера деятельности
            where i.name is NULL                                                            -- фильтрация строк (назване сферы деятельноси NULL)
        '''

In [314]:
#Результат запроса 5.4
df = pd.read_sql_query(query5_4, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1016146493.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5_4, connection)


,count
0,8419


## Задание 5.5

*Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.*

*Введите в поле ниже название этой компании так же, как оно указано в результате запроса.*

In [315]:
#Код запроса 5.5
query5_5 = f'''select
            e.name,                                                                 -- Запрос имени компании-работодателя
            count(i.name) c_iname                                                   -- Запрос количества сфер деятельности
            from public.employers e
            left join public.employers_industries ei on e.id  = ei.employer_id       
            left join public.industries i on i.id = ei.industry_id                  -- Соединение таблиц работодатель - вспом. табл, работодатель - сфера деятельности
            group by e.name                                                         -- группировка по имени компании-работодателя
            having count(i.name) = 4                                                -- фильтрация по количеству сфер деятельности
            order by 1                                                              -- сортировка по 1 столбцу
            limit 3
        '''

In [316]:
#Результат запроса 5.5
df = pd.read_sql_query(query5_5, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1613439576.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5_5, connection)


,name,c_iname
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


## Задание 5.6

*С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».*

In [317]:
#Код запроса 5.6
query5_6 = f'''select
            count(e.name) cnt                                                           -- Запрос количества названий компаний-работодателей
            from public.employers e
            left join public.employers_industries ei on e.id  = ei.employer_id       
            left join public.industries i on i.id = ei.industry_id                      -- Соединение таблиц работодатель - вспом. табл, работодатель - сфера деятельности
            group by i.name                                                             -- группировка по названиям сферы деятельности
            having i.name = 'Разработка программного обеспечения'                       -- фильтрация по ключевой фразе
        '''

In [318]:
#Результат запроса 5.6
df = pd.read_sql_query(query5_6, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\2607198655.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5_6, connection)


,cnt
0,3553


### Задание 5.7

*Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице. Должна получиться выборка такого вида (приведён пример результата для компании SberTech):*



In [319]:
#Код запроса 5.7
# million_cities - список городов - миллионников
million_cities = 'Москва', 'Санкт-Петербург', 'Новосибирск','Екатеринбург','Казань','Красноярск','Нижний Новгород','Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар','Омск','Воронеж','Пермь','Волгоград'
query5_7 = f'''with ts as(                                         -- создаём подзапрос
            select
            a.name area,                                           -- Подзапрос: название региона
            count(v.id) cnt                                        -- Подзапрос: количество вакансий
            from public.vacancies v
            left join public.areas a on v.area_id  = a.id       
            left join public.employers e on v.employer_id = e.id   -- соединение таблиц вакансия - регион, вкансия - работодатель
            where e.name = 'Яндекс'                                -- фильрация по имени компании-работодателя
            group by a.name                                        -- группировка по названию региона
            having a.name in {million_cities}                      -- фильтрация  по городам-миллионникам
            )                                               
            
            (select
            'total' as area,                                       -- запрос общего количества по отфильтрованному подзапросу
            sum(ts.cnt)
            from ts
            
            union all                                              -- объединение таблиц
            
            select
            ts.area as area,                                       -- запрос количества вакансий компании "Яндекс" в городах миллионниках
            ts.cnt
            from ts)
            order by 2
        '''

In [320]:
#Результат запроса 5.7
df = pd.read_sql_query(query5_7, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1294855927.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query5_7, connection)


,area,sum
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


## Общий вывод по блоку 5:

*После анализа рынка компаний-работодателей можно сделать следующие выводы:*

5.1. В топ 5 по количеству размещённых вакансий входят следующие           работодатели:
    - Яндекс;
    - Ростелеком;
    - Тинькофф;
    - СБЕР;
    - Газпром нефть

5.2. Наибольшее количество работодателей находится в России, Казахстане, данное распределение подтверждает распределение количества вакансий в задании 4.1.

5.3. Наибольшее количество регионов, в которых работает компания - **181**. Компания - Яндекс.

5.4. Количество работодателей, указавших свою сферу деятельности - **8419**.

5.5. На третьем месте, среди компаний, у который 4 сферы деятьности  - компания "2ГИС".

5.6. Среди работодателей, занимаются разработкой ПО - **3553**.

5.7.1. Общее количество строк - **17**.

5.7.2. Всего вакансий компании 'Яндекс' в городах миллионниках - **485**.

# 6. Предметный анализ

## Задание 6.1
*Сколько вакансий имеет отношение к данным?*

In [321]:
#Код запроса 6.1
query6_1 = f'''select
            count(v.id)                                                    -- запрос количества вакансий 
            from vacancies v
            where (v.name ilike '%данн%') or (v.name ilike '%data%')       -- фильтрация по ключевым словам
        '''

In [322]:
#Результат запроса 6.1
df = pd.read_sql_query(query6_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1387393998.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query6_1, connection)


,count
0,1771


## Задание 6.2
*Сколько есть подходящих вакансий для начинающего дата-сайентиста?*

*Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:*

- ‘data scientist’;
- ‘data science’;
- ‘исследователь данных’;
- ‘ML’ (здесь не нужно брать вакансии по HTML);
- ‘machine learning’;
- ‘машинн%обучен%’.

*В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).*

*Считаем вакансиями для специалистов уровня Junior следующие:*

- в названии есть слово “junior” или
- требуемый опыт — «Нет опыта» или
- тип трудоустройства — «Стажировка».

*В качестве ответа запишите количество вакансий, которое вернул запрос.*


In [323]:
#Код запроса 6.2
query6_2 = f'''select
            count(v.id)                                                                                                                 -- запрос количества вакансий
            from vacancies v
            where (v.name ilike '%data scientist%' or v.name ilike '%data science%' or v.name ilike '%исследователь данных%'            -- фильтрация по ключевым словам
            or v.name ilike '%ML%' or v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%')
            and v.name not ilike '%HTML%'
            and
            (v.name ilike '%junior%' or v.experience ilike '%Нет опыта%' or v.employment ilike '%Cтажировка%')
        '''

In [324]:
#Результат запроса 6.2
df = pd.read_sql_query(query6_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1671569556.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query6_2, connection)


,count
0,51


## Задание 6.3

*Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?*

*В качестве ответа запишите количество, которое вернул запрос.*

In [325]:
#Код запроса 6.3
query6_3 = f'''select
            count(v.id)                                                                                                                 -- запрос количества вакансий
            from vacancies v
            where (v.name ilike '%data scientist%' or v.name ilike '%data science%' or v.name ilike '%исследователь данных%'            -- фильтрация по ключевым словам
            or v.name ilike '%ML%' or v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%') 
            and v.name not ilike '%HTML%'
            and
            (v.key_skills ilike '%SQL%' or v.key_skills ilike '%postgres%')
        '''

In [326]:
#Результат запроса 6.3
df = pd.read_sql_query(query6_3, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1095674966.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query6_3, connection)


,count
0,229


## Задание 6.4

*С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.*

*В качестве ответа запишите количество, которое вернул запрос.*

In [327]:
#Код запроса 6.4
query6_4 = f'''select
            count(v.id)                                                                                                                 -- запрос количества вакансий
            from vacancies v
            where (v.name ilike '%data scientist%' or v.name ilike '%data science%' or v.name ilike '%исследователь данных%'            -- фильтраия по ключевым словам
            or v.name ilike '%ML%' or v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%') 
            and v.name not ilike '%HTML%'
            and
            v.key_skills ilike '%python%'
        '''

In [328]:
#Результат запроса 6.4
df = pd.read_sql_query(query6_4, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\3194521171.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query6_4, connection)


,count
0,357


## Задание 6.5

*Сколько ключевых навыков в среднем указывают в вакансиях для DS?*

*Ответ округлите до двух знаков после точки-разделителя.*



In [329]:
#Код запроса 6.5
query6_5 = f'''select
            round(avg(length(v.key_skills) - length(replace(key_skills,CHR(9),''))+1),2)                            -- запрос кол-ва символов разность длины строки с разделителм '\' и без
            from vacancies v
            where v.name ILIKE '%data scientist%'                                                                   -- фильтрация по ключевым словам
            OR v.name ILIKE '%data science%'
            OR v.name ILIKE '%исследователь данных%'
            OR (v.name  LIKE '%ML%' AND  v.name NOT iLIKE '%html%')
            OR v.name ILIKE '%machine learning%'
            OR v.name ILIKE '%машинн%обучен%'
        '''

In [330]:
#Результат запроса 6.5
df = pd.read_sql_query(query6_5, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\2843919106.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query6_5, connection)


,round
0,6.41


### Задание 6.6

*Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).*

*При решении задачи примите во внимание следующее:*

1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null).

*Чтобы избежать этой ситуации, мы воспользуемся функцией coalesce , которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос select 1 + coalesce(null, 0).*

*Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.*

In [331]:
#Код запроса 6.6
query6_6 = f'''
            select
            v.experience,                                                                                               -- запрос опыта работы (количество лет стажа)
            ROUND(AVG(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)), 0) as AVG                 -- запрос осреднённой запрвлаты
            from vacancies v 
            where (v.salary_to is not NULL or v.salary_from is not NULL)                                                -- фильтрация по ключевым словам
            and
            (
            (v.name ILIKE '%data scientist%')                       
                OR (lower(v.name) ILIKE '%data science%')
                OR (v.name ILIKE '%исследователь данных%')
                OR (v.name  LIKE '%ML%' AND  (v.name NOT iLIKE '%html%'))
                OR (v.name ILIKE '%machine learning%')
                OR (v.name ILIKE '%машинн%обучен%')
                AND (key_skills IS NOT NULL)
            )
            group by v.experience                                                                                       -- группировка по опыту
        '''

In [332]:
#Результат запроса 6.6
df = pd.read_sql_query(query6_6, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\1642566383.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query6_6, connection)


,experience,avg
0,Нет опыта,74643.0
1,От 1 года до 3 лет,145236.0
2,От 3 до 6 лет,243115.0


## Общий вывод по блоку 6:

*После предметного анализа вакансий можно сделать следующие выводы:*

6.1. Всего 1771 вакансия имеет отношение к данным.

6.2. Среди вакансий, связанных с анализом данных - **51** вакансия подходит для начинающего 'дата сиентиста'.

6.3. Среди вакансий на проффессию 'дата сиентист' - **229** вакансий требуют знание SQL.

6.4. Среди вакансий на профессию 'дата сиентист' - **357** вакансий требуют знание Python*.

6.5. В среднем, для профессии 'дата сиентист' требуется 6-7 ключевых навыков.

6.6. Средняя зарплата опытного 'дата сиентиста' составляет **243 тысячи рублей**.


## 7. Дополнительные исследования

### 1. Проверим среднюю зарплату по регионам


In [333]:
#Код запроса
query7_1 = f'''select
            a.name area,                                                                                        -- запрос региона
            ROUND(AVG(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)), 0) as AVG         -- запрос осреднённой зарплаты
            from vacancies v
            left join areas a on v.area_id = a.id                                                               -- соединение таблиц вакансия-регион
            where (v.salary_to is not NULL or v.salary_from is not NULL)                                        -- фильтрация неуказанных зарплат
            group by a.name                                                                                     -- группировка по назаванию региона
            order by avg desc                                                                                   -- группировка по средней зарплате
            limit 10
        '''

In [335]:
# Результат запроса
df = pd.read_sql_query(query7_1, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\3145620491.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query7_1, connection)


,area,avg
0,Германия,452751.0
1,Болгария,324348.0
2,Испания,300000.0
3,Черногория,288589.0
4,Литва,263019.0
5,Польша,257156.0
6,Петропавловка (Республика Бурятия),250000.0
7,Турция,235435.0
8,Кипр,223736.0
9,Кутаиси,220475.0


### 2. Проверим сколько у скольких процентов вакансий указан уровень заработной платы

In [336]:
#Код запроса
query7_2 = f'''with tab1 as                                                        -- создаем подзапрос
            (
            select *
            from vacancies v
            where v.salary_from is not NULL or v.salary_to is not NULL
            )
            
            select                                                      
            count(t.id) as сnt,                                                     -- Запрос количества вакансий, где указана ЗП из таблицы подзапроса
            round(count(t.id)::NUMERIC/count(v.id)*100) as share                    -- Запрос отношения всех вакансий к количеству в указанной ЗП
            from vacancies v
            left join tab1 t on v.id=t.id                                           -- Соединение таблицы вакансий с скоращеннйо таблицей подзапроса
            '''

In [337]:
# Результат запроса
df = pd.read_sql_query(query7_2, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\689567351.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query7_2, connection)


,сnt,share
0,24073,49.0


### 3. Проверим в какие компании требуются <Дата сиентисты>

In [338]:
query7_3 = f'''
            ((select                                                        -- создаём подзапрос
            e.name,                                                         -- имя компании-работодателя
            count(v.id) count                                               -- количество вакансий
            from vacancies v
            left join employers e on v.employer_id = e.id                   -- соединение таблица вакансия-работодатель
            where v.name ILIKE '%data scientist%'                           -- фильрация по ключевым словам                                      
            OR v.name ILIKE '%data science%'
            OR v.name ILIKE '%исследователь данных%'
            OR (v.name  LIKE '%ML%' AND  v.name NOT iLIKE '%html%')
            OR v.name ILIKE '%machine learning%'
            OR v.name ILIKE '%машинн%обучен%'
            group by e.name
            order by count(v.id) desc)
            
            union                                                           -- объединение таблиц
            
            (select                                                         -- Добавляем общее количество работодателей   
            'Всего работодателей',
            count( e.id) count
            from employers e))
            order by count desc
            '''

In [339]:
# Результат запроса
df = pd.read_sql_query(query7_3, connection)
df

C:\Users\User\AppData\Local\Temp\ipykernel_4084\2168807672.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query7_3, connection)


,name,count
0,Всего работодателей,23501
1,СБЕР,37
2,Bell Integrator,25
3,Банк ВТБ (ПАО),18
4,VK,15
...,...,...
221,Азимут Технолоджи,1
222,MNOGOGAMES,1
223,ГридПоинт Дайнамикс,1
224,билайн,1


# Общий вывод в конце по результатам анализа, имеющихся данных по вакансиям

*Наибольшее количество вакансий размещено в Москве и Санкт-Петербурге, а также в городах-миллионниках.*

*Средняя зарплата на сайте HeadHunter составляет 90 тысяч рублей*

*Наиболее распространенными работодателями я вляются крупные транснациональные компании, такие как Яндекс, Сбер, Газпромнефть*

*Наибольшая средняя зарплата в основном для вакансий в странах Европы*

*Среди всех вакансий 49% вакансий указывают зарплату*

*Среди всего количества работодателей (23501) дата сейнтисты требуются в 225 компаний*


***Для сравнительного анализа, по основным параметрам, таких как - количество вакансий, количество работодателей, уровень зарплаты, компании-лидеры, требуевые вакансии необходимо выполнить анализ базы данных сайта HeadHunter через некоторое время (1-3 года)***